In [1]:
from gurobipy import *
import pandas as pd
import numpy as np

In [2]:
Year = 1   # Do not change!

CElevts = 2
LElevts = 2
ETerminals = 2
IChina = 2
Alpha = 0.9    # Inventory deterioration rate
#Dom_P =  200   # Domestic Soybean price
Beta1 = 315.948
Beta2 = 0
Beta3 = -4.43476   # Regression coefficients
#Glo_P =  400   # Global Soybean price
Gamma1 = 117.09
Gamma2 = 0
Gamma3 = 5.6e-6  # Regression coefficients
I_last = np.array([14.7,10]) # last year inventory for each country elevator #2019

In [3]:
# Parameters
    ## from CElevts to LElevts by Trucks  e ,j
Cost_C_L = np.array([[10,7],[8,12]])
    ## from LElevts to ETerminals by Barges j,k
Cost_L_E = np.array([[50,68], [55,40]])
    ## from ETerminals to IChina by Ocean shipment k,m
Cost_E_I = np.array([[100,140],[90,88]])
    ## from CElevts to Domestic Processing Facility C^P
Cost_E_F = np.array([13,15])
    ##E Country elevators hoding cost C^H
Cost_E_H = np.array([10,10])
    ## Supply of each Country elevator
S = np.array([46e6,51e6])
    ## Demand of China each ports
D = 88e6

In [4]:
# Model
model = Model("Soybean Jun 01 2021 V12")

Academic license - for non-commercial use only - expires 2022-08-22
Using license file C:\Users\heish\gurobi.lic


In [5]:
# Vars
x = model.addVars(CElevts, LElevts, lb=0, name="x_ejt")
I = model.addVars(CElevts, lb=0, name='I_et')
f = model.addVars(CElevts, lb=0, name='f_et')
y = model.addVars(LElevts, ETerminals, lb=0, name='y_jkt')
z = model.addVars(ETerminals, IChina, lb=0, name='z_kmt')
Dp= model.addVar(vtype=GRB.CONTINUOUS, name='DomesticP')
Gp= model.addVar(vtype=GRB.CONTINUOUS, name='GlobalP')
## Slack Vars
L = model.addVar(Year, vtype=GRB.BINARY, name='L-Climate')
F = model.addVar(Year, vtype=GRB.BINARY, name='F-GOVPolicy')

In [6]:
# add constraints 2
for e in range(CElevts):
    model.addConstr(Alpha*I_last[e] + S[e] == f[e] + I[e] + quicksum(x[e,j] for j in range(LElevts)))

<ipython-input-6-55e517d125b4>:3: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  model.addConstr(Alpha*I_last[e] + S[e] == f[e] + I[e] + quicksum(x[e,j] for j in range(LElevts)))


In [7]:
# add Constraints 3
for j in range(LElevts):
    model.addConstr(quicksum(x[e,j] for e in range(CElevts)) == quicksum(y[j,k] for k in range(ETerminals)))

In [8]:
# add Constratints 4
for k in range(ETerminals):
    model.addConstr(quicksum(y[j,k] for j in range(LElevts)) == quicksum(z[k,m] for m in range(IChina)))

In [9]:
# add Constratints 5
model.addConstr(quicksum(z[k,m] for k in range(ETerminals)
                                for m in range(IChina)) <= D)

<gurobi.Constr *Awaiting Model Update*>

In [10]:
# add Constratints 6
model.addConstr(Dp == Beta1 + Beta2*L + Beta3*quicksum(I))

<gurobi.Constr *Awaiting Model Update*>

In [11]:
# add Constratints 7
model.addConstr(Gp == Gamma1 + Gamma2*F + Gamma3*quicksum(z))

<gurobi.Constr *Awaiting Model Update*>

In [12]:
# Obj
obj = LinExpr()
obj += quicksum((Gp-Cost_E_I[k,m])*z[k,m] for k in range(ETerminals) 
                                             for m in range(IChina))
obj -= quicksum(Cost_C_L[e,j]*x[e,j] for e in range(CElevts)
                                     for j in range(LElevts))
obj += quicksum((Dp-Cost_E_F[e])*f[e] for e in range(CElevts))
obj -= quicksum(Cost_E_H[e]*I[e] for e in range(CElevts))
obj -= quicksum(Cost_L_E[j,k]*y[j,k] for j in range(LElevts)
                                     for k in range(ETerminals))
model.setObjective(
    obj,
    GRB.MAXIMIZE
)

In [13]:
model.update()
model.params.NonConvex = 2
model.optimize()
model.write('model.lp')

Changed value of parameter NonConvex to 2
   Prev: -1  Min: -1  Max: 2  Default: -1
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 4 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 9 rows, 20 columns and 36 nonzeros
Model fingerprint: 0x19b21018
Model has 6 quadratic objective terms
Variable types: 18 continuous, 2 integer (2 binary)
Coefficient statistics:
  Matrix range     [6e-06, 4e+00]
  Objective range  [7e+00, 1e+02]
  QObjective range [2e+00, 2e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+02, 9e+07]
Presolve removed 0 rows and 2 columns
Presolve time: 0.00s
Presolved: 22 rows, 25 columns, 71 nonzeros
Presolved model has 6 bilinear constraint(s)
Variable types: 25 continuous, 0 integer (0 binary)

Root relaxation: objective 4.428886e+10, 15 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Nod

In [14]:
# Print solution
print('Number of variables: ' + str(model.getAttr("NumVars")))
print('Number of linear constraints: '+ str(model.getAttr("NumConstrs"))) 
print('Objective value for current solution: '+ str(model.getAttr("ObjVal"))) 

Number of variables: 20
Number of linear constraints: 9
Objective value for current solution: 44288858756.22404


In [15]:
print(x,'\n',I,'\n',f,'\n',y,'\n',z,'\n',Dp,'\n',Gp,'\n',L,'\n',F)


{(0, 0): <gurobi.Var x_ejt[0,0] (value 0.0)>, (0, 1): <gurobi.Var x_ejt[0,1] (value 46000013.23)>, (1, 0): <gurobi.Var x_ejt[1,0] (value 0.0)>, (1, 1): <gurobi.Var x_ejt[1,1] (value 41999986.77)>} 
 {0: <gurobi.Var I_et[0] (value 0.0)>, 1: <gurobi.Var I_et[1] (value 0.0)>} 
 {0: <gurobi.Var f_et[0] (value 0.0)>, 1: <gurobi.Var f_et[1] (value 9000022.229999997)>} 
 {(0, 0): <gurobi.Var y_jkt[0,0] (value 0.0)>, (0, 1): <gurobi.Var y_jkt[0,1] (value 0.0)>, (1, 0): <gurobi.Var y_jkt[1,0] (value 0.0)>, (1, 1): <gurobi.Var y_jkt[1,1] (value 88000000.0)>} 
 {(0, 0): <gurobi.Var z_kmt[0,0] (value 0.0)>, (0, 1): <gurobi.Var z_kmt[0,1] (value 0.0)>, (1, 0): <gurobi.Var z_kmt[1,0] (value 0.0)>, (1, 1): <gurobi.Var z_kmt[1,1] (value 88000000.0)>} 
 <gurobi.Var DomesticP (value 315.948)> 
 <gurobi.Var GlobalP (value 609.89)> 
 <gurobi.Var L-Climate (value 1.0)> 
 <gurobi.Var F-GOVPolicy (value 1.0)>
